# 機械学習で画像認識！
- [参考1](https://qiita.com/Yu_unI1/items/cc992918c6f940d10699)
- [参考2](https://sorabatake.jp/11731/)
- 学習モデルはいろいろ試せると思うよ

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #Google Driveをマウントする

Mounted at /content/drive


In [ ]:
# 沢山ライブラリをインポート
import numpy as np
import random #擬似乱数を生成するライブラリー
from PIL import Image, ImageOps #画像処理ライブラリー
from sklearn.metrics import accuracy_score #モデルの正解率を計算するライブラリー
import glob #ファイルパス名を取得するライブラリ
from PIL import Image, ImageOps #画像処理ライブラリー

In [ ]:
# 画像を学習用に圧縮
from google.colab.patches import cv2_imshow

labels =["A", "B", "C"] # 適当な名前でいいんよ
pathes = ["/content/drive/MyDrive/AAA", "/content/drive/MyDrive/BBB", "/content/drive/MyDrive/CCC"] # Drive上の教師データのフォルダを指定
n_classes = len(labels)
print(f"num of class = {n_classes} , classes = {labels}")

X = []
Y = []

for i in range(n_classes):
  files = glob.glob(pathes[i] + "/*.jpg")
  print(len(files))
  for file in files:
    img1 = Image.open(file)
    img = img1.resize((64,64)) # 画像サイズ(X x Y),小さいほうが基本良い
    im_array = np.ravel(np.asarray(img)) # 画像を配列に変換
    im_regularized = im_array/255. # 正規化
    X.append(im_regularized) # 画像を一つの配列にまとめる、ここが機械学習の闇
    Y.append(i)

    # 上下反転
    img1 = ImageOps.flip(img1)
    img = img1.resize((64,64)) # 画像サイズ(X x Y),小さいほうが基本良い
    im_array = np.ravel(np.asarray(img)) # 画像を配列に変換
    im_regularized = im_array/255. # 正規化
    X.append(im_regularized) # 画像を一つの配列にまとめる、ここが機械学習の闇
    Y.append(i)

    # 左右反転
    img1 = ImageOps.mirror(img1)
    img = img1.resize((64,64)) # 画像サイズ(X x Y),小さいほうが基本良い
    im_array = np.ravel(np.asarray(img)) # 画像を配列に変換
    im_regularized = im_array/255. # 正規化
    X.append(im_regularized) # 画像を一つの配列にまとめる、ここが機械学習の闇
    Y.append(i)

    # 上下左右反転
    img1 = ImageOps.mirror(img1)
    img1 = ImageOps.flip(img1)
    img = img1.resize((64,64)) # 画像サイズ(X x Y),小さいほうが基本良い
    im_array = np.ravel(np.asarray(img)) # 画像を配列に変換
    im_regularized = im_array/255. # 正規化
    X.append(im_regularized) # 画像を一つの配列にまとめる、ここが機械学習の闇
    Y.append(i)

num of class = 3 , classes = ['空', '紙', '段ボール']
50
50
50


In [ ]:
# 学習用のデータと評価用のデータを9:1で分割
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1)

In [ ]:
from sklearn.svm import LinearSVC # ライブラリのインポート、サポートベクターマシン
import xgboost as xgb # ライブラリのインポート、アンサンブル型、精度はいいけどちょい遅い

# SVMの学習
# model = LinearSVC(C=0.3, random_state=0) # インスタンスを生成
# model.fit(X_train, Y_train) # モデルの学習

# XGBoostの学習(コメント外してSVMにコメントつける)
model = xgb.XGBClassifier(n_estimators=80, max_depth=4, gamma=3) # インスタンスの生成
model.fit(X_train, Y_train) # モデルの学習

# 学習データに対する精度
print("Train :", model.score(X_train,  Y_train))

# テストデータに対する精度
print("Test :", model.score(X_test, Y_test))

Train : 0.9981481481481481
Test : 0.9666666666666667


In [ ]:
# Pickleによるモデル保存
import pickle

# モデルを保存する
# この保存先もDriveにして任意の時にモデルを読み込めるようにしたい
filename = 'finalized_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
from google.colab.patches import cv2_imshow
from google.colab import files

# 保存したモデルをロードする
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

#sample画像の前処理
image = files.upload() # ローカルからファイルをアップロード
image_name = next(iter(image))
img1 = Image.open(image_name)
img2 = img1.resize((64,64)) # 画像サイズ(X x Y),小さいほうが基本良い
img = np.array(img2)
im_array = np.ravel(np.asarray(img)) # 画像を配列に変換
im_regularized = im_array/255. # 正規化
im_regularized = im_regularized.reshape(1, -1)
# モデル推定
pred = int(loaded_model.predict(im_regularized))
print(labels[pred]) # 結果発表！

Saving 20240402223504.jpg to 20240402223504.jpg
段ボール


<ipython-input-10-cfd7d3ec0dd4>:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = int(loaded_model.predict(im_regularized))
